In [7]:
import ollama
import numpy as np
import pandas as pd
import time
import pickle
import random
import os
import requests
import json
import logging


def get_response(prompt):
    url = "https://chatapi.midjourney-vip.cn/v1/chat/completions"
    model = "gpt-4o-2024-05-13"
    payload = json.dumps({
    "model": model,
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ]})
    headers = {
    'Accept': 'application/json',
    'Authorization': 'sk-OOLfa5EgvvQNeXP03953498fE4004dB3811e10B582C2C127',
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    responses = response.text
    return responses

def is_valid_output(feature):
    return len(feature.split()) < 8 and feature != ''

def get_prompt(finetune_prompt):
    number = 0
    while True:
        response = get_response(finetune_prompt)
        # print(response)
        
        prompt_positions = [i for i in range(len(response)) if response.startswith('<PROMPT>', i)]
        
        if len(prompt_positions) < 2:
            print("Invalid output format, retrying...")
            number = number+1
            if number > 8:
                return None
            continue

        begin = prompt_positions[-2] + len('<PROMPT>"')
        end = prompt_positions[-1]

        response = response[begin:end].replace('\\n', '\n')

        begin = response.find('Task')
        end = response.rfind('Gene_name') + len('Gene_name')
        response = response[begin:end]

        return response

        
def compare_list(word_list):
    common_features = word_list[0]
    number = 0
    for lst in word_list[1:]:
        number += len(set(common_features) & set(lst))

    return number

def get_num():
    if not os.path.exists('txt/data/good_num.npy'):
        if not os.path.exists('txt/data/ori.npy'):
            with open('/home/yuantao/code/LLM/txt/prompt/prompt.txt', 'r') as f:
                prompt = f.read()
                ori_num = []
                for idx, gene_name in enumerate(gene_name_list):
                    word_list = [] 
                    for i in range(4):
                        
                        while True:
                            new_prompt = prompt.replace('Gene_name', gene_name)
                            content = ollama.generate(model='gemma2', prompt=new_prompt)['response']

                            start_pos = content.find('<begin>')
                            end_pos = content.rfind('<end>')
                            
                            if start_pos == -1 or end_pos == -1 or start_pos >= end_pos:
                                print("Invalid output format, retrying...")
                                continue

                            feature_start_idx = start_pos + len('<begin>')
                            feature_end_idx = end_pos
                            feature = content[feature_start_idx:feature_end_idx].strip()
                            

                            feature_list = feature.split(',')
                            if len(feature_list) >= MAX_FEATURES: 
                                feature_list = feature_list[:MAX_FEATURES]
                            
                            if is_valid_output(feature_list[0]):
                                feature_list = [word.strip().lower() for word in feature_list if word != '']
                                word_list.append(feature_list)
                                break
                            else:
                                print("Invalid feature")
                                print("Retrying...")

                    count = compare_list(word_list)
                    print(gene_name, count)
                    ori_num.append(count)

                print('ori_num:', ori_num)
                np.save('txt/data/ori.npy', ori_num)
                good_num = ori_num
                bad_num = ori_num
        else:
            good_num = np.load('txt/data/ori.npy')
            bad_num = np.load('txt/data/ori.npy')
    else:
        good_num = np.load('txt/data/good_num.npy')
        bad_num = np.load('txt/data/bad_num.npy')

    # print(good_num,np.mean(good_num))
    # print(bad_num,np.mean(bad_num))
    return good_num, bad_num

In [8]:
prompt = open("txt/prompt/prompt.txt", "r").read()
fine_tune_prompt = open("txt/prompt/fine_tune_prompt.txt", "r").read()
cancer_name = 'pan-cancer'
prompt = prompt.replace('Cancer_name', cancer_name)

# gene_name_list = open(r'/home/yuantao/code/LLM/txt/data/node_names.txt','r',encoding='utf-8').read().split('\n')
# gene_name_list = random.sample(gene_name_list, 20)
gene_name_list = ['ENSG00000179627,ZBTB42', 'ENSG00000154269,ENPP3', 'ENSG00000164096,C4orf3', 'ENSG00000188191,PRKAR1B', 'ENSG00000131779,PEX11B', 'ENSG00000151164,RAD9B', 'ENSG00000162670,BRINP3', 'ENSG00000171848,RRM2', 'ENSG00000174132,FAM174A', 'ENSG00000103546,SLC6A2', 'ENSG00000115762,PLEKHB2', 'ENSG00000142082,SIRT3', 'ENSG00000128335,APOL2', 'ENSG00000134897,BIVM', 'ENSG00000167941,SOST', 'ENSG00000119630,PGF', 'ENSG00000102575,ACP5', 'ENSG00000135919,SERPINE2', 'ENSG00000092847,AGO1', 'ENSG00000100302,RASD2']
print(gene_name_list)

model = 'gemma2'
MAX_FEATURES = 6
get_num()

['ENSG00000179627,ZBTB42', 'ENSG00000154269,ENPP3', 'ENSG00000164096,C4orf3', 'ENSG00000188191,PRKAR1B', 'ENSG00000131779,PEX11B', 'ENSG00000151164,RAD9B', 'ENSG00000162670,BRINP3', 'ENSG00000171848,RRM2', 'ENSG00000174132,FAM174A', 'ENSG00000103546,SLC6A2', 'ENSG00000115762,PLEKHB2', 'ENSG00000142082,SIRT3', 'ENSG00000128335,APOL2', 'ENSG00000134897,BIVM', 'ENSG00000167941,SOST', 'ENSG00000119630,PGF', 'ENSG00000102575,ACP5', 'ENSG00000135919,SERPINE2', 'ENSG00000092847,AGO1', 'ENSG00000100302,RASD2']


(array([ 5,  4,  1,  6,  5,  3,  3, 10,  0,  1,  4,  3,  0,  0,  5,  5,  2,
         8,  5,  3]),
 array([ 5,  4,  1,  6,  5,  3,  3, 10,  0,  1,  4,  3,  0,  0,  5,  5,  2,
         8,  5,  3]))

In [14]:
while True:
    get_num()
    good_prompt = open("txt/prompt/good_prompt.txt", "r").read()
    bad_prompt = open("txt/prompt/bad_prompt.txt", "r").read()
    good_num, bad_num = get_num()
    good_mean = np.mean(good_num)
    bad_mean = np.mean(bad_num)

    # input_GPT_prompt = fine_tune_prompt.replace('Good_PROMPT', good_prompt).replace('Good_NUM', str(good_mean)).replace('Bad_PROMPT', bad_prompt).replace('Bad_NUM', str(bad_mean))
    # prompt = get_prompt(input_GPT_prompt)
    prompt = open("/home/yuantao/code/LLM/txt/prompt/good_prompt.txt", "r").read()
    prompt = prompt.replace('Cancer_name', cancer_name)
    print(prompt)

    GPT_num = []
    for idx, gene_name in enumerate(gene_name_list):
        word_list = [] 
        for i in range(4):
            retry = 0
            while True:
                if retry > 8:
                    print('retry too many times')
                    break
                new_prompt = prompt.replace('Gene_name', gene_name)
                content = ollama.generate(model='gemma2', prompt=new_prompt)['response']

                start_pos = content.find('<begin>')
                end_pos = content.rfind('<end>')
                
                if start_pos == -1 or end_pos == -1 or start_pos >= end_pos:
                    print("Invalid output format, retrying...")
                    retry += 1
                    continue

                feature_start_idx = start_pos + len('<begin>')
                feature_end_idx = end_pos
                feature = content[feature_start_idx:feature_end_idx].strip()
                

                feature_list = feature.split(',')
                if len(feature_list) >= MAX_FEATURES: 
                    feature_list = feature_list[:MAX_FEATURES]
                
                if is_valid_output(feature_list[0]):
                    feature_list = [word.strip().lower() for word in feature_list if word != '']
                    word_list.append(feature_list)
                    break
                else:
                    retry += 1
                    print("Invalid feature")
                    print("Retrying...")


        count = compare_list(word_list)
        GPT_num.append(count)
        print(gene_name, count)
    print('GPT_num:', GPT_num)
    GPT_mean = np.mean(GPT_num)

    print('GPT_num:', np.mean(GPT_num))
    with open('txt/prompt/result.txt','a') as f:
        f.write('good_num: ' + str(good_mean) + '\n')
        f.write('list: ' + str(good_num) + '\n')
        f.write('bad_num: ' + str(bad_mean) + '\n')
        f.write('list: ' + str(bad_num) + '\n')
        f.write('GPT_num: ' + str(GPT_mean) + '\n')
        f.write('list: ' + str(GPT_num) + '\n')
        f.write('\n')

    if good_mean < GPT_mean:
        print('GPT is better than good')
        bad_num = good_num
        bad_mean = good_mean

        good_mean = GPT_mean
        good_num = GPT_num

        bad_prompt = open("txt/prompt/good_prompt.txt", "r").read()
        good_prompt = prompt

    elif bad_mean < GPT_mean < good_mean:
        print('GPT is better than bad')
        bad_num = GPT_num
        bad_mean = GPT_mean

        bad_prompt = prompt
        good_prompt = open("txt/prompt/good_prompt.txt", "r").read()

    else:
        print('GPT is worse than good and bad')

    with open('txt/prompt/good_prompt.txt', 'w') as f:
        f.write(good_prompt)
    with open('txt/prompt/bad_prompt.txt', 'w') as f:
        f.write(bad_prompt)

    np.save('txt/data/good_num.npy', good_num)
    np.save('txt/data/bad_num.npy', bad_num)
    


Analysis:
Upon comparison, the stable prompt exhibits a slight edge over the suboptimal prompt in terms of repetition accuracy and output stability (number of repeated words on 20 genes being 4.4 vs. 4.2). The stability in the prompt primarily arises from the additional specificity and iterative reinforcement of vocabulary consistency.
   
Key areas of stability in the stable prompt include:
1. Explicit instructions on gene description first and vocabulary summarization next.
2. Clear emphasis on aligning descriptive vocabulary directly with described functions and roles.
3. Specific mention of avoiding overly common terms, ensuring relevance to cancer biology.
4. Instruction to iteratively review vocabulary for alignment, enhancing consistency.
    
To further boost the number of repetitions and stability, consider:
1. Emphasizing the use of specific cancer-related concepts and technical terms.
2. Reinforcing unique biological features related to gene functions to reduce variability. 

KeyboardInterrupt: 